In [1]:
import numpy as np
import scipy as sp
from sklearn.neighbors import NearestNeighbors
import time
from scipy.spatial import KDTree
from sklearn.neighbors import BallTree
from dis_utils import *
import numpy as np
import pandas as pd
from tools import *
from sklearn.neighbors import NearestNeighbors
import time
from scipy.spatial import KDTree
from sklearn.neighbors import BallTree

In [2]:
#class for UCB

# entry amount we choose once in a pull
pull_once=50

class UCB:
    def __init__(self,arm_array,delta=0.1,n_best_arms=10,mode="rough",k=5,random_list=[]):
        '''
        arm_array is the data matrix we have (in type of np.array)
        delta is the error rate
        n_best_arms is the best arms amount we want
        mode is the calculating result wanted, if it needs to be accurate
        k is the rough range, we can almost make sure every points lays in k*n_best_arms neighbors
        '''
        self.random_list=random_list
        self.pull_time=0
        self.delta=delta
        self.wanted_best_amount=n_best_arms
        self.real_best_amount=0
        self.row=len(arm_array)
        self.col=len(arm_array[0])
        self.data=arm_array
        self.MAX_PULL=int(self.col/pull_once)
        self.threshold=np.inf
        self.best_list=np.array([])
        self.lidx= np.arange(self.row)
#         self.arm_states: "pull sum sum_square ucb lcb status 
#         ( 1 means should be used 
#           0 means this one should be picked as one of the best arms
#           -1 means this one is too high to pick (won't be the best arms) )"
        self.arm_states=np.tile(np.array([0,0,0,np.inf,-np.inf,1]), (self.row,1))
        temp_pull=np.random.randint(0,self.col-1,10*pull_once)
        for x in range(self.row):
            #take some entries in the arm to ensure the initial variance value of arm is roughly right
            l=self.data[x,temp_pull]
            self.arm_states[x][1]=np.sum(l)
            self.arm_states[x][2]=np.sum(l**2)
        
    def print_arm_states(self,x):
        print(self.arm_states[x])
        return
    
    def pull_arm(self,x,cell_select):
        '''
        pull one specific arm
        '''
        dim=self.col
        if(self.arm_states[x][-1]==1):
            l=self.data[x][cell_select]
            self.arm_states[x][0]+=1
            
            #record the sum and sum of square to speed up calculating variance.
            self.arm_states[x][1]+=np.sum(l)
            self.arm_states[x][2]+=np.sum(l**2)
            
            pull_time_now=self.arm_states[x][0]*pull_once+500
            var=self.arm_states[x][2]/pull_time_now-(self.arm_states[x][1]/pull_time_now)**2
            
            #for the variance, using the np.log(x)+1 to reduce the influence of too large or small variance
            #adding 1e-6 to variance too avoid log(0)
            #fixed variance here means the fixed gap between ucb and lcb, which will make the period as comparision based on mean
            
            CB=cal_CB(self.arm_states[x][0],(np.log(var+1)+1)/100,self.delta,self.row,self.MAX_PULL,'fit')
            mean=self.arm_states[x][1]/self.arm_states[x][0]/pull_once
            self.arm_states[x][3]=CB+mean
            lowest_cb=max(0,-CB+mean)
            self.arm_states[x][4]=lowest_cb
            
            #if the lcb higher than the thershold, then we will label this arm as "impossible to be the best arms"
            if (lowest_cb>self.threshold):
                self.arm_states[x][-1]=-1
                self.lidx=self.lidx[self.lidx!=x]
            return
        return 
    
    def pull_everyarm(self):
        '''
        pull every arms once
        store the best arms in self.best_list
        '''
        if(self.wanted_best_amount==self.real_best_amount):
            #if we've already choosed a preset amount of best arms, then break the program
            print('completed')
            return
        #randomly choose the pull index
        if(len(self.lidx)==0):
            return
        
        temp_pull=self.random_list[pull_time]
        self.pull_time+=1
        for x in range(self.row):
            #pull every arms once
            self.pull_arm(x,temp_pull)
            
            #choosing threshold
        ucb=self.arm_states[:,3]
#         #1. pick a fixed value
#         self.threshold=1
#         #2. pick a fixed arm
#         self.threshold=self.arm_states[1][3]
#         #3. pick a fixed order position arms （50 here）
#         self.threshold=np.partition(ucb, 49)[49]
#         #3.1. pick a lowest ucb
#        self.threshold = np.min(ucb[np.where(ucb>0.01)])*0.8



        #4.randomly choose 20 (can be changed) arms and using the average of their ucb as the threshold.
#         print(len(self.lidx))
#        For the method 4, set attenuation coefficient to 0.7-0.8 is okay for the result.
#        In common condition, if we want k neighbors, our result will be 50%-70% correct.
#        But usually more than 70% of them will be in the 2k neighbors
        self.threshold =np.average(self.arm_states[np.random.choice(self.lidx,20),3])*0.75
    
    
        #if don't set the thershold above, only picking the arm satisfying the lowest condition, it'll be very low efficient
        self.judge()
        return
    
    def judge(self):
        '''
        judge if there's any confidence range satisfying the lowest : ucb < others' lcbs
        '''
        # this function is to verify if there's some best arms existing in matrix
        
        ucb=arm_assem.arm_states[:,3][self.lidx]
#         print('ucb:',ucb[:10])
        lcb=arm_assem.arm_states[:,4][self.lidx]
        if(len(lcb)<2):
            return
        if(np.min(ucb)<np.min(lcb[np.where(lcb!=np.min(lcb))])):
#             print(np.min(ucb),np.min(lcb[np.where(lcb!=np.min(lcb))]))
            #if the lowest ucb lower than the 2nd lowest lcb, then this should be the best
            idx=np.argmin(ucb)
            self.arm_states[idx][-1]=0
            self.best_list=np.append(self.best_list,idx)
            self.real_best_amount+=1
            self.lidx=self.lidx[self.lidx!=idx]
    def get_knn(self):
        return

In [3]:
X=read_mat('original')
X=np.array(X,dtype=int)

In [7]:
target=10
nbrs = NearestNeighbors(n_neighbors=500, algorithm='auto').fit(X)
A=nbrs.kneighbors(X, return_distance=False)
ls=A[target]
rl=6
idx=np.arange(rl)
Correction_Rate1=np.zeros(rl)
Correction_Rate2=np.zeros(rl)
Correction_Rate3=np.zeros(rl)
Length=np.zeros(rl)
dirt={'index':idx,
     'Correction_Rate1':Correction_Rate1,
      'Correction_Rate2':Correction_Rate2,
      'Correction_Rate3':Correction_Rate3,
      'Length':Length
     }
M=pd.DataFrame(dirt)

for ii in range(rl):
    if(ii==rl-1):
        A1=np.array(M['Correction_Rate1'])
        idxA1=np.where(A1<1)
        lengthidx=len(idxA1[0])-1
        A1=np.array(M['Correction_Rate1'])[idxA1]        
        A2=np.array(M['Correction_Rate2'])[idxA1]
        A3=np.array(M['Correction_Rate3'])[idxA1]
        A4=np.array(M['Length'])[idxA1]
        M['Correction_Rate1'][rl-1]=np.sum(A1)/lengthidx
        M['Correction_Rate2'][rl-1]=np.sum(A2)/lengthidx
        M['Correction_Rate3'][rl-1]=np.sum(A3)/lengthidx
        M['Length'][ii]=np.sum(A4)/lengthidx
        break
#     print('list & truth:',lidx,ground_truth1)
    arm_assem=UCB(abs(X-X[target]),delta=0.1)
    for i in range(100):
        arm_assem.pull_everyarm()
    lidx=np.where(arm_assem.arm_states[:,-1]!=-1)
    lengthlidx=len(lidx[0])
    
    if(lengthlidx>100):
        print('set to NaN')
        M['Correction_Rate1'][ii]=np.nan
        M['Correction_Rate2'][ii]=np.nan
        M['Correction_Rate3'][ii]=np.nan
        M['Length'][ii]=np.nan
        continue
    ground_truth1=ls[:lengthlidx]
    ground_truth2=ls[:2*lengthlidx]
    ground_truth3=ls[:5*lengthlidx]

    M['Correction_Rate1'][ii]=list_verify(lidx,ground_truth1)

    M['Correction_Rate2'][ii]=list_verify(lidx,ground_truth2)
    M['Correction_Rate3'][ii]=list_verify(lidx,ground_truth3)
    M['Length'][ii]=len(lidx[0])
M

C:\Users\rossoneri\AppData\Local\Temp\ipykernel_6424\2951289219.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['Correction_Rate1'][ii]=list_verify(lidx,ground_truth1)
C:\Users\rossoneri\AppData\Local\Temp\ipykernel_6424\2951289219.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['Correction_Rate2'][ii]=list_verify(lidx,ground_truth2)
C:\Users\rossoneri\AppData\Local\Temp\ipykernel_6424\2951289219.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,index,Correction_Rate1,Correction_Rate2,Correction_Rate3,Length
0,0,0.500000,0.625000,0.875000,8.0
1,1,0.625000,0.687500,0.875000,16.0
2,2,0.710526,0.973684,1.000000,38.0
3,3,0.760870,0.978261,1.000000,46.0
4,4,0.843750,0.984375,0.984375,64.0
5,5,0.688029,0.849764,0.946875,34.4


In [4]:
def list_verify(a1,a2):
    same_list=np.intersect1d(a1,a2, assume_unique=False, 
                  return_indices=False)
    return len(same_list)/len(a1[0])

In [92]:
nbrs = NearestNeighbors(n_neighbors=500, algorithm='auto').fit(X)
A=nbrs.kneighbors(X, return_distance=False)
ls=A[15]
ls

array([  15, 1342,   44,   55, 1347, 1439, 1484,  155, 1335, 1451,   43,
         84,  218,  987,   40,  236, 1042,  398, 1509,  214, 1467, 1027,
        449, 1447, 1549, 1613,  941, 1522,  148,  876,   60, 1650, 1414,
        217, 1346,  406,  283,  260, 1311, 1102, 1442, 1077,  241,  913,
       1401,  215,   63,  727, 1553, 1524, 1475, 1036,  673,  712, 1670,
       1634, 1209,  779,  277,  349,  147,  785,  689,  208, 1919,  509,
        799, 1881, 1413,  591, 1240, 1787, 1130, 1819,  288, 1393,  431,
        626,  499,  886,  229, 1154, 1624, 1763,  216,  893, 1625,  301,
       1357,  979, 1095,  835, 1610,  998, 1380, 1621, 1887, 1883,  824,
       1252, 1547, 1575,  818, 1744,  612, 1379, 1100, 1778, 1748,  670,
       1431,  695, 1437,  397, 1699, 1680, 1754,  902, 1398,  374, 1641,
        318,  206, 1707,  324,  403, 1015, 1060, 1061,  451, 1568,  379,
        249, 1114,  484, 1730, 1751,  724,   18, 1833,  897, 1470, 1057,
        790, 1785, 1086, 1008, 1640, 1852, 1139, 13

In [17]:
rl=len(X)
idx=np.arange(rl)
Correction_Rate1=np.zeros(rl)
Correction_Rate2=np.zeros(rl)
Correction_Rate3=np.zeros(rl)
dirt={'index':idx,
     'Correction_Rate1':Correction_Rate1,
      'Correction_Rate2':Correction_Rate2,
      'Correction_Rate3':Correction_Rate3
     }
M=pd.DataFrame(dirt)

In [26]:
dirt['Correction_Rate1'][0]=1
M=pd.DataFrame(dirt)
print(M)
M['Correction_Rate1'][0]=np.nan
print(M)

    index  Correction_Rate1  Correction_Rate2  Correction_Rate3  Length
0       0               1.0               0.0               0.0     0.0
1       1               0.0               0.0               0.0     0.0
2       2               0.0               0.0               0.0     0.0
3       3               0.0               0.0               0.0     0.0
4       4               0.0               0.0               0.0     0.0
5       5               0.0               0.0               0.0     0.0
6       6               0.0               0.0               0.0     0.0
7       7               0.0               0.0               0.0     0.0
8       8               0.0               0.0               0.0     0.0
9       9               0.0               0.0               0.0     0.0
10     10               0.0               0.0               0.0     0.0
11     11               0.0               0.0               0.0     0.0
12     12               0.0               0.0               0.0 

C:\Users\rossoneri\AppData\Local\Temp\ipykernel_2200\3275530239.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M['Correction_Rate1'][0]=np.nan


In [132]:
M.to_csv('M_time100_atten1.3_p7_3.csv', index=True,mode='a')

In [5]:
def plot_error_bars(ucb_list, lcb_list):
    if len(ucb_list) != len(lcb_list):
        raise ValueError("UCB and LCB should share the same list length")
    values = [(ucb + lcb) / 2 for ucb, lcb in zip(ucb_list, lcb_list)]
    error_lower = [value - lcb for value, lcb in zip(values, lcb_list)]
    error_upper = [ucb - value for value, ucb in zip(values, ucb_list)]
    points = np.arange(len(values))
    plt.figure()
    plt.errorbar(points, values, yerr=[error_lower, error_upper], fmt='o', capsize=5, label='Points with LCB and UCB')
    plt.xlabel('Point Index')
    plt.ylabel('Value')
    plt.title('Error Bar Plot with LCB and UCB')
    plt.legend()
    plt.grid(True)
    plt.show()
fixed_var=100000000
def cal_CB(pull_num,var,delta,number_of_arms,MAX_PULL,mode='fit'):
    delta=delta/number_of_arms/(MAX_PULL)
    if mode=='fit':
        return np.sqrt(2*var*np.log(2/delta)/pull_num)
    if mode=='fixed':
        return np.sqrt(2*fixed_var*np.log(2/delta)/pull_num)
    return

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
np.set_printoptions(threshold=np.inf)

In [74]:
fixed_var=100000000
def cal_CB(pull_num,var,delta,number_of_arms,MAX_PULL,mode='fit'):
    delta=delta/number_of_arms/(MAX_PULL)
    if mode=='fit':
        return np.sqrt(2*var*np.log(2/delta)/pull_num)
    if mode=='fixed':
        return np.sqrt(2*fixed_var*np.log(2/delta)/pull_num)
    return
def plot_error_bars(ucb_list, lcb_list):
    if len(ucb_list) != len(lcb_list):
        raise ValueError("UCB and LCB should share the same list length")
    values = [(ucb + lcb) / 2 for ucb, lcb in zip(ucb_list, lcb_list)]
    error_lower = [value - lcb for value, lcb in zip(values, lcb_list)]
    error_upper = [ucb - value for value, ucb in zip(values, ucb_list)]
    points = np.arange(len(values))
    plt.figure()
    plt.errorbar(points, values, yerr=[error_lower, error_upper], fmt='o', capsize=5, label='Points with LCB and UCB')
    plt.xlabel('Point Index')
    plt.ylabel('Value')
    plt.title('Error Bar Plot with LCB and UCB')
    plt.legend()
    plt.grid(True)
    plt.show()
#class for UCB
pull_once=50
class UCB:
    def __init__(self,arm_array,delta=0.1,n_best_arms=10,mode="rough",max_iter=50,k=5):
        self.delta=delta
        self.wanted_best_amount=n_best_arms
        self.real_best_amount=0
        self.row=len(arm_array)
        self.col=len(arm_array[0])
        self.data=arm_array
        self.MAX_PULL=int(self.col/pull_once)
        self.threshold=np.inf
        self.best_list=np.array([])
        self.lidx= np.arange(self.row) 
#         self.arm_states: "pull sum sum_square ucb lcb status 
#         ( 1 means should be used 
#           0 means this one should be picked as one of the best arms
#           -1 means this one is too high to pick (won't be the best arms) )"
        self.arm_states=np.tile(np.array([0,0,0,np.inf,-np.inf,1]), (self.row,1))
        temp_pull=np.random.randint(0,self.col-1,10*pull_once)
        for x in range(self.row):
            l=self.data[x,temp_pull]
            self.arm_states[x][1]=np.sum(l)
            self.arm_states[x][2]=np.sum(l**2)
        
    def print_arm_states(self,x):
        print(self.arm_states[x])
        return
    
    def pull_arm(self,x,cell_select):
        dim=self.col
        if(self.arm_states[x][-1]==1):
            l=self.data[x][cell_select]
            self.arm_states[x][0]+=1
            self.arm_states[x][1]+=np.sum(l)
            self.arm_states[x][2]+=np.sum(l**2)
            
            pull_time_now=self.arm_states[x][0]*pull_once
            var=self.arm_states[x][2]/pull_time_now-(self.arm_states[x][1]/pull_time_now)**2
            CB=cal_CB(self.arm_states[x][0],(np.log(var+1e-6)+1)/100,self.delta,self.row,self.MAX_PULL,'fit')
#             pull_time_now=self.arm_states[x][0]*pull_once+500
#             var=self.arm_states[x][2]/pull_time_now-(self.arm_states[x][1]/pull_time_now)**2
#             CB=cal_CB(self.arm_states[x][0],(np.log(var+1)+1)/100,self.delta,self.row,self.MAX_PULL,'fit')
            mean=self.arm_states[x][1]/self.arm_states[x][0]/pull_once
            #print('CB and mean:', CB, mean)
            self.arm_states[x][3]=CB+mean
            lowest_cb=max(0,-CB+mean)
            self.arm_states[x][4]=lowest_cb
#             print('threshold:', self.threshold)
            if (lowest_cb>self.threshold):
                self.arm_states[x][-1]=-1
                self.lidx=self.lidx[self.lidx!=x]
            return
        return 
    
    def pull_everyarm(self):
        if(self.wanted_best_amount==self.real_best_amount):
            print('completed')
            return
        temp_pull=np.random.randint(0,self.col-1,pull_once)
        for x in range(self.row):
            self.pull_arm(x,temp_pull)
            
            #choosing threshold
        ucb=self.arm_states[:,3]
#         print(ucb)
#         #1. pick a fixed value
#         self.threshold=1
#         #2. pick a fixed arm
#         self.threshold=self.arm_states[1][3]
#         #3. pick a fixed order position arms （50 here）
#         self.threshold=np.partition(ucb, 49)[49]
#         #3.1. pick a lowest ucb
#         self.threshold = np.min(ucb[np.where(ucb>0.01)])*1.3



        #4.randomly choose 20 (can be changed) arms and using the average of their ucb as the threshold.
#         print(len(self.lidx))
#        For the method 4, set attenuation coefficient to 0.7-0.8 is okay for the result.
#        In common condition, if we want k neighbors, our result will be 50%-70% correct.
#        But usually more than 70% of them will be in the 2k neighbors
        self.threshold =np.average(self.arm_states[np.random.choice(self.lidx,20),3])*0.75

        return
    
    def judge(self):
        print('yes, you')
        ucb=arm_assem.arm_states[:,3]
        lcb=arm_assem.arm_states[:,4]
        if(np.min(ucb)<np.min(lcb[np.where(lcb!=np.min(lcb))])):
            idx=np.argmin(ucb)
            self.arm_states[idx][-1]=0
            np.append(self.best_list,idx)
            self.real_best_amount+=1
    def get_knn(self):
        return